# Import necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [3]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sales_train_validation.csv')

### Note: This Step is for the extraction of California State and Foods category 

In [4]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'HOBBIES')]

In [5]:
INPUT_DIR_2 = '/content/drive/MyDrive/Colab Notebooks/List of Product ID according to 4 demand patterns/California/Hobbies/2-year'
list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID_2_Year_Data.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID_2_Year_Data.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID_2_Year_Data.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID_2_Year_Data.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

# User-defined functions to calculate Metrics and Croston_TSB algorithm

In [6]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

In [7]:
def Croston_TSB(ts,extra_periods=1,alpha=0.4,beta=0.4):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), probability(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
# Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1/(1 + first_occurence)
    f[0] = p[0]*a[0]
                 
    # Create all the t+1 forecasts
    for t in range(0,cols): 
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = beta*(1) + (1-beta)*p[t]  
        else:
            a[t+1] = a[t]
            p[t+1] = (1-beta)*p[t]       
        f[t+1] = p[t+1]*a[t+1]
        
    # Future Forecast
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

# Set value for parameters

In [ ]:
list_params_alpha = [round(item, 1) for item in list(np.arange(0.1, 1, 0.1))]
list_params_beta = [round(item, 1) for item in list(np.arange(0.1, 1, 0.1))]

In [8]:
start_train_date = '2014-04-11'
end_train_date = '2016-04-10'
n_pred_days = 14
num_train_needed = 100
validation_training_ratio = 0.95

# Pipeline for Croston_TSB

In [ ]:
def CrostonTSB_output_all_params(sales_pattern, list_params_alpha, list_params_beta, 
                                 start_train_date, end_train_date, n_pred_days, 
                                 num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    train_data_first_part = train_data.iloc[:int(len(train_data)*validation_training_ratio), :]
    train_data_second_part = train_data.iloc[int(len(train_data)*validation_training_ratio):, :]
    
    df_result_all_params = pd.DataFrame()
    for product in list_pattern_py:
        train_first = train_data_first_part[product]
        train_second = train_data_second_part[product]
        for i in list_params_alpha:
            for j in list_params_beta:
                predictions = list()
                history = [x for x in train_first[num_train_needed*-1:]]
                for t in range(len(train_second)):
                    yhat = Croston_TSB(history, extra_periods=1, alpha = i, beta = j)['Forecast'].iloc[-1]
                    predictions.append(yhat)
                    history.append(train_second[t])
                df_result_temp = pd.DataFrame({'Product': [product for count in range(len(train_second))],
                                               'Actual Data': train_second,
                                               'Forecast': predictions,
                                               'Alpha': [i for count_i in range(len(train_second))],
                                               'Beta': [j for count_j in range(len(train_second))]})
                df_result_all_params = df_result_all_params.append(df_result_temp, ignore_index=True)
            
    return df_result_all_params

In [ ]:
sales_intermittent

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
6,HOBBIES_1_007_CA_1_validation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,1,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9707,HOBBIES_2_145_CA_4_validation,HOBBIES_2_145,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9708,HOBBIES_2_146_CA_4_validation,HOBBIES_2_146,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9709,HOBBIES_2_147_CA_4_validation,HOBBIES_2_147,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9710,HOBBIES_2_148_CA_4_validation,HOBBIES_2_148,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
sales_intermittent.iloc[:500]

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
6,HOBBIES_1_007_CA_1_validation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,1,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3110,HOBBIES_1_063_CA_2_validation,HOBBIES_1_063,HOBBIES_1,HOBBIES,CA_2,CA,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3111,HOBBIES_1_064_CA_2_validation,HOBBIES_1_064,HOBBIES_1,HOBBIES,CA_2,CA,0,0,0,0,...,0,2,0,1,1,0,2,1,2,4
3112,HOBBIES_1_065_CA_2_validation,HOBBIES_1_065,HOBBIES_1,HOBBIES,CA_2,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3115,HOBBIES_1_068_CA_2_validation,HOBBIES_1_068,HOBBIES_1,HOBBIES,CA_2,CA,3,0,0,4,...,1,0,0,0,0,0,0,0,2,0


In [ ]:
sales_intermittent.iloc[500:1000]

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
3117,HOBBIES_1_070_CA_2_validation,HOBBIES_1_070,HOBBIES_1,HOBBIES,CA_2,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3118,HOBBIES_1_072_CA_2_validation,HOBBIES_1_072,HOBBIES_1,HOBBIES,CA_2,CA,0,0,0,0,...,1,1,0,0,0,0,0,1,0,0
3119,HOBBIES_1_073_CA_2_validation,HOBBIES_1_073,HOBBIES_1,HOBBIES,CA_2,CA,1,0,0,2,...,0,4,2,0,0,0,0,1,1,0
3120,HOBBIES_1_074_CA_2_validation,HOBBIES_1_074,HOBBIES_1,HOBBIES,CA_2,CA,4,1,0,3,...,2,3,2,0,1,0,6,0,0,3
3121,HOBBIES_1_075_CA_2_validation,HOBBIES_1_075,HOBBIES_1,HOBBIES,CA_2,CA,3,0,0,0,...,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6226,HOBBIES_1_133_CA_3_validation,HOBBIES_1_133,HOBBIES_1,HOBBIES,CA_3,CA,0,0,0,0,...,0,0,0,0,0,0,0,1,2,0
6228,HOBBIES_1_135_CA_3_validation,HOBBIES_1_135,HOBBIES_1,HOBBIES,CA_3,CA,0,0,0,0,...,0,3,2,0,1,0,0,2,1,0
6229,HOBBIES_1_136_CA_3_validation,HOBBIES_1_136,HOBBIES_1,HOBBIES,CA_3,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6230,HOBBIES_1_137_CA_3_validation,HOBBIES_1_137,HOBBIES_1,HOBBIES,CA_3,CA,1,2,2,4,...,0,0,1,1,1,1,0,0,0,0


In [ ]:
sales_intermittent.iloc[1000:1500]

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
6232,HOBBIES_1_139_CA_3_validation,HOBBIES_1_139,HOBBIES_1,HOBBIES,CA_3,CA,0,0,0,0,...,0,0,1,2,2,1,0,0,0,3
6234,HOBBIES_1_141_CA_3_validation,HOBBIES_1_141,HOBBIES_1,HOBBIES,CA_3,CA,0,2,1,1,...,3,2,3,2,2,1,2,4,3,2
6236,HOBBIES_1_143_CA_3_validation,HOBBIES_1_143,HOBBIES_1,HOBBIES,CA_3,CA,0,1,0,0,...,0,1,2,0,0,0,0,0,0,0
6237,HOBBIES_1_144_CA_3_validation,HOBBIES_1_144,HOBBIES_1,HOBBIES,CA_3,CA,0,0,0,0,...,0,1,1,0,1,0,1,1,0,0
6239,HOBBIES_1_146_CA_3_validation,HOBBIES_1_146,HOBBIES_1,HOBBIES,CA_3,CA,0,0,0,1,...,1,1,3,2,1,5,2,2,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9341,HOBBIES_1_201_CA_4_validation,HOBBIES_1_201,HOBBIES_1,HOBBIES,CA_4,CA,2,4,3,2,...,0,2,0,1,2,0,1,2,1,0
9343,HOBBIES_1_203_CA_4_validation,HOBBIES_1_203,HOBBIES_1,HOBBIES,CA_4,CA,0,0,0,0,...,1,0,1,1,0,1,0,0,1,0
9344,HOBBIES_1_204_CA_4_validation,HOBBIES_1_204,HOBBIES_1,HOBBIES,CA_4,CA,0,0,0,0,...,0,1,0,2,0,0,1,1,1,0
9345,HOBBIES_1_205_CA_4_validation,HOBBIES_1_205,HOBBIES_1,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
sales_intermittent.iloc[1500:]

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
9347,HOBBIES_1_207_CA_4_validation,HOBBIES_1_207,HOBBIES_1,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
9348,HOBBIES_1_208_CA_4_validation,HOBBIES_1_208,HOBBIES_1,HOBBIES,CA_4,CA,0,0,0,0,...,0,1,1,1,0,0,0,2,1,0
9349,HOBBIES_1_209_CA_4_validation,HOBBIES_1_209,HOBBIES_1,HOBBIES,CA_4,CA,0,0,4,0,...,0,0,0,0,0,0,0,0,0,0
9350,HOBBIES_1_210_CA_4_validation,HOBBIES_1_210,HOBBIES_1,HOBBIES,CA_4,CA,0,0,0,0,...,0,1,0,1,2,0,0,1,0,2
9351,HOBBIES_1_211_CA_4_validation,HOBBIES_1_211,HOBBIES_1,HOBBIES,CA_4,CA,0,0,0,0,...,5,7,7,4,2,4,1,1,1,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9707,HOBBIES_2_145_CA_4_validation,HOBBIES_2_145,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9708,HOBBIES_2_146_CA_4_validation,HOBBIES_2_146,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9709,HOBBIES_2_147_CA_4_validation,HOBBIES_2_147,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9710,HOBBIES_2_148_CA_4_validation,HOBBIES_2_148,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
start = timer()

pattern_df_result_all_params = CrostonTSB_output_all_params(sales_intermittent.iloc[1500:],
                                                          list_params_alpha,
                                                          list_params_beta,
                                                          start_train_date,
                                                          end_train_date, 
                                                          n_pred_days,
                                                          num_train_needed,
                                                          validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 14.433801241916665 minutes


In [ ]:
pattern_df_result_all_params

,Product,Actual Data,Forecast,Alpha,Beta
0,HOBBIES_1_207_CA_4_validation,0.0,0.057011,0.1,0.1
1,HOBBIES_1_207_CA_4_validation,0.0,0.051310,0.1,0.1
2,HOBBIES_1_207_CA_4_validation,0.0,0.046179,0.1,0.1
3,HOBBIES_1_207_CA_4_validation,0.0,0.041561,0.1,0.1
4,HOBBIES_1_207_CA_4_validation,0.0,0.037405,0.1,0.1
...,...,...,...,...,...
905089,HOBBIES_2_149_CA_4_validation,0.0,0.900009,0.9,0.9
905090,HOBBIES_2_149_CA_4_validation,0.0,0.090001,0.9,0.9
905091,HOBBIES_2_149_CA_4_validation,0.0,0.009000,0.9,0.9
905092,HOBBIES_2_149_CA_4_validation,0.0,0.000900,0.9,0.9


In [ ]:
pattern_df_result_all_params.to_csv('CrostonTSB_Intermittent_1500_End_All_Parameters_Hyperparameter_Tuning_Data.csv')

In [ ]:
df_res_1 = pd.read_csv('/content/CrostonTSB_Intermittent_0_500_All_Parameters_Hyperparameter_Tuning_Data.csv', index_col=0)
df_res_2 = pd.read_csv('/content/CrostonTSB_Intermittent_500_1000_All_Parameters_Hyperparameter_Tuning_Data.csv', index_col=0)
df_res_3 = pd.read_csv('/content/CrostonTSB_Intermittent_1000_1500_All_Parameters_Hyperparameter_Tuning_Data.csv', index_col=0)
df_res_4 = pd.read_csv('/content/CrostonTSB_Intermittent_1500_End_All_Parameters_Hyperparameter_Tuning_Data.csv', index_col=0)

In [ ]:
pattern_df_result_all_params = pd.concat([df_res_1, df_res_2, df_res_3, df_res_4], axis=0)

In [ ]:
pattern_df_result_all_params

,Product,Actual Data,Forecast,Alpha,Beta
0,HOBBIES_1_001_CA_1_validation,2.0,0.296313,0.1,0.1
1,HOBBIES_1_001_CA_1_validation,0.0,0.410458,0.1,0.1
2,HOBBIES_1_001_CA_1_validation,0.0,0.369412,0.1,0.1
3,HOBBIES_1_001_CA_1_validation,0.0,0.332471,0.1,0.1
4,HOBBIES_1_001_CA_1_validation,0.0,0.299224,0.1,0.1
...,...,...,...,...,...
905089,HOBBIES_2_149_CA_4_validation,0.0,0.900009,0.9,0.9
905090,HOBBIES_2_149_CA_4_validation,0.0,0.090001,0.9,0.9
905091,HOBBIES_2_149_CA_4_validation,0.0,0.009000,0.9,0.9
905092,HOBBIES_2_149_CA_4_validation,0.0,0.000900,0.9,0.9


# Summay all metrics

In [9]:
def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [ ]:
start = timer()

df_result_metrics_all_params = pattern_df_result_all_params.groupby(['Product', 'Alpha', 'Beta']).apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 5.881734759983344 minutes


In [ ]:
df_result_metrics_all_params

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOBBIES_1_001_CA_1_validation,0.1,0.1,0.893266,124.2107,162.3729,65.7844,0.570698,0.666009
1,HOBBIES_1_001_CA_1_validation,0.1,0.2,0.907946,126.2520,163.1403,65.5560,0.580077,0.681826
2,HOBBIES_1_001_CA_1_validation,0.1,0.3,0.911101,126.6907,164.5734,65.6523,0.582093,0.696933
3,HOBBIES_1_001_CA_1_validation,0.1,0.4,0.915254,127.2682,166.3304,66.3461,0.584746,0.712547
4,HOBBIES_1_001_CA_1_validation,0.1,0.5,0.927198,128.9289,168.4445,68.2715,0.592376,0.729642
...,...,...,...,...,...,...,...,...,...
145957,HOBBIES_2_149_CA_4_validation,0.9,0.5,0.957571,196.8341,199.6710,98.9541,0.159595,0.327060
145958,HOBBIES_2_149_CA_4_validation,0.9,0.6,0.966311,198.6305,199.8363,99.4877,0.161052,0.339463
145959,HOBBIES_2_149_CA_4_validation,0.9,0.7,0.970739,199.5408,199.9390,99.8110,0.161790,0.352852
145960,HOBBIES_2_149_CA_4_validation,0.9,0.8,0.972506,199.9040,199.9862,99.9573,0.162084,0.367529


# Check what products has unexpected metrics result

In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['MASE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
810,FOODS_1_008_CA_1_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
811,FOODS_1_008_CA_1_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
812,FOODS_1_008_CA_1_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
813,FOODS_1_008_CA_1_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
814,FOODS_1_008_CA_1_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...
289975,FOODS_3_826_CA_3_validation,0.9,0.5,inf,inf,200.0,NaN,2.805754e-10,9.853484e-10
289976,FOODS_3_826_CA_3_validation,0.9,0.6,inf,inf,200.0,NaN,3.649518e-13,1.453276e-12
289977,FOODS_3_826_CA_3_validation,0.9,0.7,inf,inf,200.0,NaN,7.469419e-17,3.333996e-16
289978,FOODS_3_826_CA_3_validation,0.9,0.8,inf,inf,200.0,NaN,5.115764e-22,2.540772e-21


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['WMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
810,FOODS_1_008_CA_1_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
811,FOODS_1_008_CA_1_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
812,FOODS_1_008_CA_1_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
813,FOODS_1_008_CA_1_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
814,FOODS_1_008_CA_1_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...
289975,FOODS_3_826_CA_3_validation,0.9,0.5,inf,inf,200.0,NaN,2.805754e-10,9.853484e-10
289976,FOODS_3_826_CA_3_validation,0.9,0.6,inf,inf,200.0,NaN,3.649518e-13,1.453276e-12
289977,FOODS_3_826_CA_3_validation,0.9,0.7,inf,inf,200.0,NaN,7.469419e-17,3.333996e-16
289978,FOODS_3_826_CA_3_validation,0.9,0.8,inf,inf,200.0,NaN,5.115764e-22,2.540772e-21


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['SMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
810,FOODS_1_008_CA_1_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.0,0.0
811,FOODS_1_008_CA_1_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.0,0.0
812,FOODS_1_008_CA_1_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.0,0.0
813,FOODS_1_008_CA_1_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.0,0.0
814,FOODS_1_008_CA_1_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...
289165,FOODS_3_823_CA_1_validation,0.9,0.5,NaN,NaN,NaN,NaN,0.0,0.0
289166,FOODS_3_823_CA_1_validation,0.9,0.6,NaN,NaN,NaN,NaN,0.0,0.0
289167,FOODS_3_823_CA_1_validation,0.9,0.7,NaN,NaN,NaN,NaN,0.0,0.0
289168,FOODS_3_823_CA_1_validation,0.9,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['MAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
810,FOODS_1_008_CA_1_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
811,FOODS_1_008_CA_1_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
812,FOODS_1_008_CA_1_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
813,FOODS_1_008_CA_1_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
814,FOODS_1_008_CA_1_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...
289975,FOODS_3_826_CA_3_validation,0.9,0.5,inf,inf,200.0,NaN,2.805754e-10,9.853484e-10
289976,FOODS_3_826_CA_3_validation,0.9,0.6,inf,inf,200.0,NaN,3.649518e-13,1.453276e-12
289977,FOODS_3_826_CA_3_validation,0.9,0.7,inf,inf,200.0,NaN,7.469419e-17,3.333996e-16
289978,FOODS_3_826_CA_3_validation,0.9,0.8,inf,inf,200.0,NaN,5.115764e-22,2.540772e-21


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['MAE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


# Metrics Statistics

In [ ]:
list_metrics = ['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE']

In [ ]:
for mts in list_metrics:
    print('Percentage of unexpected values of', mts, 'is: {}'.format(df_result_metrics_all_params[df_result_metrics_all_params[mts].isin([np.nan, np.inf, -np.inf])].Product.nunique() / df_result_metrics_all_params.Product.nunique() * 100), "%")

Percentage of unexpected values of MASE is: 21.365149833518313 %
Percentage of unexpected values of WMAPE is: 21.365149833518313 %
Percentage of unexpected values of SMAPE is: 18.756936736958934 %
Percentage of unexpected values of MAPE is: 21.365149833518313 %
Percentage of unexpected values of MAE is: 0.0 %
Percentage of unexpected values of RMSE is: 0.0 %


### Filter all rows that have unexpected metrics values

In [ ]:
df_result_metrics_all_params = df_result_metrics_all_params[~df_result_metrics_all_params.isin([np.nan, np.inf, -np.inf]).any(1)]

### Get MEAN metrics value of each Alpha

In [ ]:
df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()

MASE       WMAPE       SMAPE       MAPE       MAE      RMSE
Alpha Beta                                                                 
0.1   0.1   0.990360  153.202725  167.649179  70.100100  0.519054  0.707739
      0.2   0.976764  151.334931  168.384269  70.899103  0.519524  0.720327
      0.3   0.973842  151.137518  169.226049  71.658254  0.522310  0.734914
      0.4   0.973624  151.321909  170.175861  72.505086  0.525849  0.750860
      0.5   0.974775  151.678045  171.265151  73.519678  0.530036  0.768331
...              ...         ...         ...        ...       ...       ...
0.9   0.5   0.982908  152.104082  173.053726  77.137267  0.553726  0.812440
      0.6   0.985896  152.725045  174.131268  78.408667  0.558492  0.833804
      0.7   0.989671  153.438870  175.347254  79.812392  0.563949  0.857642
      0.8   0.994261  154.248796  176.746104  81.349793  0.570172  0.884524
      0.9   0.999383  155.124392  178.394178  82.985103  0.576923  0.915168

[81 rows x 6 columns]

In [ ]:
for mts in list_metrics:
    print('The optimum Alpha and Beta based on Mean', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()[mts].idxmin()))

The optimum Alpha and Beta based on Mean MASE is: (0.3, 0.4)
The optimum Alpha and Beta based on Mean WMAPE is: (0.4, 0.3)
The optimum Alpha and Beta based on Mean SMAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Mean MAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Mean MAE is: (0.1, 0.1)
The optimum Alpha and Beta based on Mean RMSE is: (0.1, 0.1)


### Get MEDIAN metrics value of each Alpha

In [ ]:
df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()

MASE     WMAPE     SMAPE     MAPE       MAE      RMSE
Alpha Beta                                                           
0.1   0.1   0.922858  151.0168  182.3314  74.2400  0.462175  0.604844
      0.2   0.929432  151.8586  182.6414  74.0666  0.457231  0.621342
      0.3   0.934726  151.0149  182.8537  73.2757  0.452255  0.634842
      0.4   0.941448  150.4394  183.2175  72.8950  0.453460  0.652578
      0.5   0.946752  150.6397  184.1994  72.9403  0.451178  0.672245
...              ...       ...       ...      ...       ...       ...
0.9   0.5   0.962788  153.0017  184.8744  76.3936  0.454386  0.686811
      0.6   0.967988  153.5776  185.6607  78.2658  0.455901  0.709576
      0.7   0.972195  154.3509  186.4358  79.9617  0.458424  0.730707
      0.8   0.973135  154.0642  187.5606  81.2943  0.459797  0.754659
      0.9   0.977838  154.3289  188.5032  83.1215  0.461876  0.787204

[81 rows x 6 columns]

In [ ]:
for mts in list_metrics:
    print('The optimum Alpha and Beta based on Median of', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()[mts].idxmin()))

The optimum Alpha and Beta based on Median of MASE is: (0.1, 0.1)
The optimum Alpha and Beta based on Median of WMAPE is: (0.1, 0.6)
The optimum Alpha and Beta based on Median of SMAPE is: (0.6, 0.1)
The optimum Alpha and Beta based on Median of MAPE is: (0.1, 0.4)
The optimum Alpha and Beta based on Median of MAE is: (0.3, 0.5)
The optimum Alpha and Beta based on Median of RMSE is: (0.1, 0.1)


# Run CrostonTSB model after decide best paramaters

In [10]:
best_alpha = 0.3
best_beta = 0.4

In [11]:
def CrostonTSB_test_data(best_alpha, best_beta, sales_pattern,
                         start_train_date, end_train_date, 
                         n_pred_days, num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    
    df_result_best_params = pd.DataFrame()
    for product in list_pattern_py:
        train_product = train_data[product]
        test_product = test_data[product]
        predictions = list()
        history = [x for x in train_product[num_train_needed*-1:]]
        for t in range(len(test_product)):
            yhat = Croston_TSB(history, extra_periods=1, alpha=best_alpha, beta = best_beta)['Forecast'].iloc[-1]
            predictions.append(yhat)
            history.append(test_product[t])
        df_result_temp = pd.DataFrame({'Product': [product for count in range(len(test_product))],
                                        'Actual Data': test_product,
                                        'Forecast': predictions})
        df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True)
            
    return df_result_best_params

In [12]:
start = timer()

pattern_df_result_best_params = CrostonTSB_test_data(best_alpha, best_beta, sales_intermittent, start_train_date, end_train_date, n_pred_days, num_train_needed, validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.4708697511500001 minutes


In [13]:
df_result_final = pattern_df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

In [14]:
df_result_final

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOBBIES_1_001_CA_1_validation,0.960742,206.9290,184.5038,76.5024,0.443419,0.508882
1,HOBBIES_1_001_CA_2_validation,0.985050,136.3915,166.0642,63.8110,0.681958,0.845230
2,HOBBIES_1_001_CA_3_validation,0.891135,85.3053,110.2561,44.2948,1.096782,1.527223
3,HOBBIES_1_001_CA_4_validation,1.075497,154.4304,175.6159,68.6523,0.661845,0.844152
4,HOBBIES_1_002_CA_1_validation,inf,inf,200.0000,NaN,0.105866,0.198213
...,...,...,...,...,...,...,...
1797,HOBBIES_2_147_CA_3_validation,0.827707,160.4477,192.4240,92.6076,0.573028,0.732746
1798,HOBBIES_2_147_CA_4_validation,NaN,NaN,NaN,NaN,0.000000,0.000000
1799,HOBBIES_2_148_CA_1_validation,1.021552,154.0187,187.6586,85.0391,0.550067,0.730745
1800,HOBBIES_2_148_CA_4_validation,0.931934,100.3621,199.9999,99.9998,0.071687,0.267261


In [15]:
df_result_final.to_csv('CrostonTSB_Intermittent_Test_Data.csv')